In [4]:
import os, sys 
os.environ["CUDA_VISIBLE_DEVICES"] = '2'
import pandas as pd 
import numpy as np 
import tensorflow as tf
from scipy.misc import imresize
from keras.preprocessing.image import load_img
from matplotlib import pyplot as plt

import warnings
warnings.simplefilter('ignore', category=DeprecationWarning)
%matplotlib inline
import matplotlib.pyplot as plt

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [31]:
def preprocess(file_name):
    x = load_img(file_name)
    x = imresize(x, (128,128), 'nearest')
    x = x / 255
    x = x.reshape((1,) + x.shape)
    return x

def load_graph(model_name):
    #graph = tf.Graph()
    graph = tf.get_default_graph()
    graph_def = tf.GraphDef()
    with open(model_name, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)
    return graph

def predict(imArr, graph=tf.get_default_graph(), in_name='import/input_1', out_name='import/k2tfout_0'):
    input_operation = graph.get_operation_by_name(in_name)
    output_operation = graph.get_operation_by_name(out_name)
    with tf.Session() as sess:
        result = sess.run(output_operation.outputs[0],
                      {input_operation.outputs[0]: imArr})
    return result

def predict_logits(imArr, graph=tf.get_default_graph(), in_name='import/input_1', out_name='import/dense_2/BiasAdd'):
    input_operation = graph.get_operation_by_name(in_name)
    output_operation = graph.get_operation_by_name(out_name)
    with tf.Session() as sess:
        result = sess.run(output_operation.outputs[0],
                      {input_operation.outputs[0]: imArr})
    return result

def get_gradient_with_y_true(imArr, y_true, graph=tf.get_default_graph(), in_name='import/input_1', out_name='import/dense_2/BiasAdd'):
    input_operation = graph.get_operation_by_name(in_name)
    output_operation = graph.get_operation_by_name(out_name)
    y_true = np.expand_dims(y_true, axis=0)
    
    y_true_tf = tf.constant(y_true, shape=y_true.shape, dtype='float64')
    y_pred_tf = output_operation.outputs[0]
    loss = tf.losses.softmax_cross_entropy(y_true_tf, y_pred_tf)
    
    grad_tf = tf.gradients(loss,[input_operation.outputs[0]])
    with tf.Session() as sess:
        grad_x = sess.run(grad_tf,
                         {input_operation.outputs[0]: imArr})
    grad_x = np.asarray(grad_x)
    grad_x = grad_x[0]
    return grad_x

def get_gradient_plain(imArr, graph=tf.get_default_graph(), in_name='import/input_1', out_name='import/dense_2/BiasAdd'):
    input_operation = graph.get_operation_by_name(in_name)
    output_operation = graph.get_operation_by_name(out_name)
    y_pred_tf = output_operation.outputs[0]
    one_hot_pred_tf = tf.one_hot(tf.nn.top_k(y_pred_tf).indices, tf.shape(y_pred_tf)[1])[0]
    loss = tf.losses.softmax_cross_entropy(one_hot_pred_tf, y_pred_tf)
    
    grad_tf = tf.gradients(loss,[input_operation.outputs[0]])
    grad_sign_tf = tf.sign(grad_tf)
    with tf.Session() as sess:
        grad_x = sess.run(grad_sign_tf,
                         {input_operation.outputs[0]: imArr})
    grad_x = np.asarray(grad_x)
    grad_x = grad_x[0]
    return grad_x

def plot_and_predict(imArr):
    prediction = predict(imArr)
    print(prediction)
    prediction2 = predict_logits(imArr)
    print(prediction2)
    plt.figure()
    plt.imshow(imArr[0])
    plt.show()
    
def softmax_T(x, T):
    x = np.asarray(x)
    e_x = np.exp(x/T)
    return e_x / e_x.sum()

In [37]:
def ODIN_score(file_name, epsilon=0.0012, T=400):
    imArr = preprocess(file_name)
    grad_x = get_gradient_plain(imArr)
    x_mod = imArr-epsilon*grad_x
    logit_pred_x_mod = predict_logits(x_mod)
    pred_x_mod = softmax_T(logit_pred_x_mod, T)
    print("image {}".format(file_name))
    print("Original score: {}".format(predict(imArr)))
    print("ODIN score: {}".format(pred_x_mod))
    

In [8]:
model_name = 'x.pb'
graph = load_graph(model_name)

In [28]:
fn = ''

imArr = preprocess(fn)
grad_x = get_gradient_plain(imArr)

In [29]:
predict(imArr-0.0012*grad_x)

array([1.0000000e+00, 1.9697435e-09], dtype=float32)

In [ ]:
fn = 'x.jpg'
ODIN_score(fn, epsilon=0.05)
plt.figure()
plt.imshow(preprocess(fn)[0])
plt.show()
fn = '/home/y.jpg'
ODIN_score(fn, epsilon=0.05)
plt.figure()
plt.imshow(preprocess(fn)[0])
plt.show()

In [ ]:
okDir = 'x/valid/OK'
im_paths = [os.path.join(okDir, im) for im in os.listdir(okDir)]
im_paths = np.random.permutation(im_paths)

for i, p in enumerate(im_paths):
    print(i)
    ODIN_score(p, epsilon=0.5)
    plt.figure()
    plt.imshow(preprocess(p)[0])
    plt.show()
    if i > 100:
        break

In [ ]:
nokDir = '/y/NOK'
im_paths = [os.path.join(nokDir, im) for im in os.listdir(nokDir)]
im_paths = np.random.permutation(im_paths)

for i, p in enumerate(im_paths):
    print(i)
    ODIN_score(p, epsilon=0.5)
    plt.figure()
    plt.imshow(preprocess(p)[0])
    plt.show()
    if i > 100:
        break

In [ ]:
paths = ['x.jpg', 'y.jpg']
for i, p in enumerate(paths):
    print(i)
    ODIN_score(p, epsilon=0.5)
    plt.figure()
    plt.imshow(preprocess(p)[0])
    plt.show()
    if i > 100:
        break

In [ ]:
fn = '.jpg'
ODIN_score(fn, epsilon=0.5)
fn = '.jpg'
ODIN_score(fn, epsilon=0.5)
fn = '/home../.jpg'
ODIN_score(fn, epsilon=0.5)

In [4]:
model_name = '/home/...pb'
graph = load_graph(model_name)

In [63]:
file_name = '/home/.jpg'
imArr = preprocess(file_name)
y_true = [1,0]
print(predict(imArr))
grad_x = get_gradient(imArr, y_true)
print(np.sum(gradient))

/home/ref1pal/anaconda2/envs/mlEnv/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


[0.09136726 0.9086327 ]
28.958494


In [ ]:
print("Predict imArr")
plot_and_predict(imArr)
print("Predict imArr+0.0012*grad_x")
plot_and_predict(imArr+0.0012*grad_x)
print("Predict imArr-0.0012*grad_x")
plot_and_predict(imArr-0.0012*grad_x)

In [74]:
softmax([-4.4521017, -2.155048 ])

array([0.09136727, 0.90863273])

In [86]:
print(softmax([-4.4521017, -2.155048 ]))
print(softmax_T(np.array([-4.4521017, -2.155048 ]), 1))

[0.09136727 0.90863273]
[0.09136727 0.90863273]


In [79]:
def softmax_T(x, T):
    x = np.asarray(x)
    e_x = np.exp(x/T)
    return e_x / e_x.sum()
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x)
    return e_x / e_x.sum()

In [40]:
in_name='import/input_1'
out_name='import/dense_2/BiasAdd'
y_true = [1,0]
input_operation = graph.get_operation_by_name(in_name)
output_operation = graph.get_operation_by_name(out_name)
y_true = np.expand_dims(y_true, axis=0)
y_true_tf = tf.constant(y_true, shape=y_true.shape, dtype='float64')
y_pred_tf = output_operation.outputs[0]
loss = tf.losses.softmax_cross_entropy(y_true_tf, y_pred_tf)
with tf.Session(graph=graph) as sess:
    result2 = sess.run(loss, 
                     {input_operation.outputs[0]: imArr})
print(result2)

2.392868
